In [285]:
import random
import numpy as np
import pandas as pd
from pandas import *
import matplotlib.pyplot as plt
import math
from datetime import datetime, date, timedelta
from scipy.stats.stats import spearmanr 
from scipy import stats
import re
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
import pickle
import us
plt.style.use(['seaborn-darkgrid'])
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

### Reading and reformatting data

In [286]:
hp = read_pickle('HousePollwithWinner_Final.pkl')
hp['year'] = hp.date.dt.to_period('Y')
# Create unique index for race name (district, state, year)
hp['race_name'] = hp['race_name'] + ', ' + hp.set_index('year').index.strftime('%Y')

sp = read_pickle('SenatePollwithWinner_Final.pkl')
sp['year'] = sp.date.dt.to_period('Y')
sp['race_name'] = sp['race_name'] + ', ' + sp.set_index('year').index.strftime('%Y')

gp = read_pickle('GovernorPollwithWinner_Final.pkl')
gp['year'] = gp.date.dt.to_period('Y')
gp['race_name'] = gp['race_name'] + ', ' + gp.set_index('year').index.strftime('%Y')

### Output for FEC/Polls model
* This is what most people should be here for

In [287]:
# Spits out average spread per candidate for congressional, senatorial, and gubernatorial races
# Sorry for the ugly code -- this function is a bunch of disjoint pieces repurposed :)
def format_that_shit(r_df):
    race_names = r_df['race_name'].unique()
    for i in race_names:
        r_df[i] = r_df['race_name'].isin([i]).astype('int')
    
    spread0 = []
    for i in race_names:
        t_df = r_df[r_df[i] == 1]
        # There are entries still marked as <BLANK>...
        t_df = t_df.replace('<BLANK>',np.nan)
        t_df['election_date'] = pd.to_datetime(t_df['election_date'])
        spread0.append(t_df[['race_name','state','poll_results','poll_leader','poll_spread','date','winner','election_date']])
    race_dict0 = dict(zip(race_names, spread0))
    
    spread1 = []
    for i in race_names:
        date = pd.DataFrame(race_dict0[i]['date']).reset_index(drop=True)
        res = pd.concat([pd.DataFrame.from_dict(i,orient='index').T for i in race_dict0[i]['poll_results']]).reset_index(drop=True)
        w_date = pd.concat([date,res], axis=1)
        w_date['date'] = to_datetime(w_date['date'])
        w_date = w_date.set_index('date')  
        w_date = w_date.groupby('date').mean()
        w_date.loc[race_dict0[i]['election_date'].reset_index(drop=True)[0]] = np.nan
        spread1.append(w_date)
    race_dict1 = dict(zip(race_names, spread1)) 
    
    spread2 = []
    for i in race_names:
        vals = race_dict1[i].mean().sort_values()
        k = vals-vals.max()
        poll = pd.DataFrame(k.replace(0,vals.max()-vals[-2:-1][0]))
        spread2.append(poll)
    spread_dict = dict(zip(race_names, spread2))
    
    candidates_rows = []
    for i in race_names:
        row = spread_dict[i]
        row['race_name'] = i
        candidates_rows.append(row)
    candidates = pd.concat(candidates_rows)
    # Splitting full race name into separate race name and year columns
    candidates = concat([candidates.reset_index()[['index',0]], pd.DataFrame(candidates['race_name'].str.split(',').tolist(), columns = ['race_name','year'])], axis = 1)
    candidates.columns = ['name', 'spread', 'race_name', 'year']
    candidates = candidates.set_index('name')
    
    return candidates

In [282]:
sen = format_that_shit(sp)
hou = format_that_shit(hp)
gov = format_that_shit(gp)

In [289]:
# Example of output
sen.head()

spread       race_name   year
name                                    
ferre  -30.808081  florida senate   2010
greene -15.946970  florida senate   2010
meek   -14.974747  florida senate   2010
crist   -3.977273  florida senate   2010
rubio    3.977273  florida senate   2010

In [283]:
# Pickle everything
sen.to_pickle('sen_avg_spread.pkl')
hou.to_pickle('house_avg_spread.pkl')
gov.to_pickle('gov_avg_spread.pkl')

### Building the model

In [274]:
# Create new columns to subset by race name
# This isn't especially pythonic but I'm past caring
for i in hp['race_name'].unique():
    hp[i] = hp['race_name'].isin([i]).astype('int')

# Create a dictionary with key values being races and value being a df of all the info we need moving forward
lis = []
for i in hp['race_name'].unique():
    df = hp[hp[i] == 1]
    # There are entries still marked as <BLANK>...
    df = df.replace('<BLANK>',np.nan)
    df['election_date'] = pd.to_datetime(df['election_date'])
    lis.append(df[['race_name','state','poll_results','poll_leader','poll_spread','date','winner','election_date']])
race_dict = dict(zip(hp['race_name'].unique(), lis))

In [277]:
# Example from race_dict
race_dict['pennsylvania 12th district special election, 2010']

race_name state  \
0   pennsylvania 12th district special election, 2010    PA   
6   pennsylvania 12th district special election, 2010    PA   
7   pennsylvania 12th district special election, 2010    PA   
9   pennsylvania 12th district special election, 2010    PA   
10  pennsylvania 12th district special election, 2010    PA   
11  pennsylvania 12th district special election, 2010    PA   

                      poll_results poll_leader  poll_spread       date winner  \
0   {'burns': 44.0, 'critz': 41.0}       burns          3.0 2010-04-20  critz   
6   {'burns': 46.0, 'critz': 40.0}       burns          6.0 2010-05-01  critz   
7   {'critz': 40.0, 'burns': 46.0}       burns          6.0 2010-05-01  critz   
9   {'critz': 44.0, 'burns': 38.0}       critz          6.0 2010-05-12  critz   
10  {'burns': 38.0, 'critz': 44.0}       critz          6.0 2010-05-12  critz   
11  {'critz': 47.0, 'burns': 48.0}       burns          1.0 2010-05-17  critz   

   election_date  
0     2010-11-02  
6     2010-11-02  
7     2010-11-02  
9     2010-11-02  
10    2010-11-02  
11    2010-11-02

In [278]:
# From the dictionaries within poll results (for specific elections) create a time series of poll data (formatted as df)
# The last value in this df is the date of the election
# Put these time series in a dictionary with the key being race name
rd = []
for i in hp['race_name'].unique():
    date = pd.DataFrame(race_dict[i]['date']).reset_index(drop=True)
    res = pd.concat([pd.DataFrame.from_dict(i,orient='index').T for i in race_dict[i]['poll_results']]).reset_index(drop=True)
    w_date = pd.concat([date,res], axis=1)
    w_date['date'] = to_datetime(w_date['date'])
    w_date = w_date.set_index('date')  
    w_date = w_date.groupby('date').mean()
    w_date.loc[race_dict[i]['election_date'].reset_index(drop=True)[0]] = np.nan
    rd.append(w_date)
race_dict_updated = dict(zip(hp['race_name'].unique(), rd))    

In [103]:
### MODEL METHODOLOGY BRIEFLY EXPLAINED HERE ###

# Final prediction on the day of the election
# Exponential moving average on past three rate of changes in polls, multiplied by days from the last poll until the election.
# This product is added to the most recent poll to output a prediction

# Predictions are in a dictionary with the key being race name
rd_final = []
for i in hp['race_name'].unique():
    polling_diff = race_dict_final[i].reset_index().diff()
    polling_diff['date'] = polling_diff['date'].dt.days
    polling_diff = polling_diff.set_index('date')
    final_polling = polling_diff.div(polling_diff.index, axis=0).ewm(span=3,min_periods=1).mean().reset_index(drop=True)[-1:].multiply(polling_diff.index[-1:],axis=0).reset_index(drop=True)
    rd_final.append(final_polling)
race_dict_final = dict(zip(hp['race_name'].unique(), rd_final)) 

In [105]:
# Unbelievably innacurate
race_dict_final['pennsylvania 12th district special election, 2010']

burns      critz
0  162.415584  73.306494

In [104]:
# If we have less than three polls we output nan
race_dict_final2['alaska at-large district, 2018']

galvin  young
0     NaN    NaN

### Results
* Plenty of nans (case where we don't have an election date and/or we have <3 total polls taken)
* Most values that aren't nan are wrong, mostly because there's often a huge gap between the last poll and the date of the election
* For example, in PA District 12 the six polls taken are on average 10 days apart, but the actual election is 6 months from the last poll!
* Very small sample sizes and predictions far outside our date range is a reciple for disaster (even with this ultra-simple model) 
* **This makes accurate prediction/extrapolation next to impossible**

In [110]:
# Conversion of the pretty dict into an ugly df
# Demonstrative of my points above

pd.DataFrame.from_dict(race_dict_final2, orient='index')

0
pennsylvania 12th district special election, 2010          burns      critz
0  162.415584  73.306494
new hampshire 1st district, 2010                        guinta  shea-porter
0 -2.357714    -4.275143
new hampshire 2nd district, 2010                           bass    kuster  swett
0 -38.393226  8....
maryland 1st district, 2010                                harris   kratovil
0  37.721519  37.721519
hawaii 1st district special election, 2010              djou  case  hanabusa
0   NaN   NaN       NaN
south carolina 4th district, 2010                     gowdy  inglis  lee  thomas  jeffrey
0    Na...
north carolina 2nd district, 2010                     ellmers  etheridge  rose
0      NaN        ...
minnesota 6th district, 2010                          anderson  bachmann    clark
0       NaN   0...
nevada 3rd district, 2010                                       heck     titus
0  0.560192 -0.104884
virginia 5th district, 2010                                   hurt  perriello
0 -8.722852  -0.858313
virginia 9th district, 2010                                  boucher  griffith
0 -0.919601  1.735991
new mexico 1st district, 2010                                 barela  heinrich
0 -7.671958 -5.677249
kentucky 6th district, 2010                                 chandler      barr
0  3.275862  3.275862
north carolina 11th district, 2010                                  shuler  miller
0     NaN     NaN
new york 23rd district, 2010                                       owens  doheny
0  1.034483     0.0
new york 25th district, 2010                                  maffei   buerkle
0  1.806452 -0.516129
pennsylvania 3rd district, 2010                           kelly  dahlkemper
0 -14.792057    2.346939
pennsylvania 10th district, 2010                              marino    carney
0  0.961156  1.252936
pennsylvania 11th district, 2010                         barletta  kanjorski
0 -42.747079  12.815946
pennsylvania 12th district, 2010                               burns     critz
0 -2.576271  0.966102
west virginia 3rd district, 2010                                  rahall  maynard
0     NaN      NaN
new york 20th district, 2010                                       murphy  gibson
0  12.9375 -4.3125
washington 3rd district, 2010                               herrera           heck
0  566404.4520...
kentucky 3rd district, 2010                                         yarmuth  lally
0     12.0  -30.0
washington 8th district, 2010                               reichert   delbene
0 -5.412698  2.677249
washington 2nd district, 2010                                 koster    larsen
0 -3.555556  4.740741
maine 2nd district, 2010                                     michaud  levesque
0 -2.080952 -1.990476
maine 1st district, 2010                                     pingree  scontras
0  2.395152 -9.039394
california 20th district, 2010                                          costa  vidak
0    NaN    NaN
michigan 9th district, 2010                                 raczkowski  peters
0         NaN     NaN
...                                                                                              ...
ohio 1st district, 2018                                           chabot  pureval
0     0.0      0.0
oklahoma 5th district, 2018                                           russell  horn
0      NaN   NaN
pennsylvania 10th district, 2018                             perry    scott
0  80272.291667  59278.0
florida 16th district, 2018                                   buchanan  shapiro
0   53362.8  35575.2
arizona 2nd district, 2018                            kirkpatrick  peterson
0          NaN       NaN
virginia 2nd district, 2018                                        taylor   luria
0 -34357.0 -2021.0
california 10th district, 2018                                     harder  denham
0 -12703.0     0.0
michigan 11th district, 2018                               stevens       epstein
0  24253.090909 ...
north carolina 13th district, 2018                         